In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
# loadDataFrom = '../data/EP_30.txt'
loadDataFrom = '../data/EP1.01.txt'
storeTensorsTo = "processedData/channelSlectedNoisyIEEE9616192"
columns = ['id', 'event', 'device', 'channel', 'code', 'size', 'data']
channelToConsider = pd.Series(['P7', 'P8', 'T7', 'T8'])
dF = pd.read_csv(filepath_or_buffer=loadDataFrom, sep='\s+', names=columns)
dF.head(5)

,id,event,device,channel,code,size,data
0,67635,67635,EP,AF3,6,260,"4395.384615,4382.564102,4377.435897,4387.17948..."
1,67636,67635,EP,F7,6,260,"4489.230769,4475.384615,4474.358974,4486.66666..."
2,67637,67635,EP,F3,6,260,"4538.461538,4528.717948,4524.615384,4526.15384..."
3,67638,67635,EP,FC5,6,260,"4207.692307,4205.641025,4200.512820,4194.35897..."
4,67639,67635,EP,T7,6,260,"4497.948717,4498.461538,4494.871794,4497.94871..."


In [3]:
cleanDf = dF.drop(['id', 'device'], axis=1)
cleanDf['size'] = pd.to_numeric(cleanDf['size'])
cleanDf['event'] = pd.to_numeric(cleanDf['event'])
cleanDf['code'] = pd.to_numeric(cleanDf['code'])
minSize = cleanDf['size'].min()
minSize

136

In [4]:
events = cleanDf['event'].drop_duplicates().to_numpy()
numEvents = len(events)
targets = []
eventTensors = []

In [5]:
for event in events:
    eventGroup = cleanDf.loc[cleanDf['event'] == event]
    eventData = []
    skip = False
    if channelToConsider.isin(eventGroup['channel']).all():
        for channel in channelToConsider:
            channelData = eventGroup.loc[eventGroup['channel'] == channel]['data'].values[0]
            channelData = np.fromstring(channelData, dtype=np.float, sep=',')[:250]
            if channelData.shape[0] < 250:
                skip = True
                break
            eventData.append(channelData)
        if skip:
            print('Skipping event: {}, due to data points less than 250'.format(event))
            continue
        eventTensors.append(torch.tensor(np.vstack(eventData)))
        targets.append(eventGroup.iloc[0]['code'])
    else:
        print("Skipping event: {}, due to lack of required channel's data".format(event))

Skipping event: 67958, due to data points less than 250
Skipping event: 67961, due to data points less than 250
Skipping event: 69134, due to data points less than 250
Skipping event: 71205, due to data points less than 250
Skipping event: 75672, due to data points less than 250
Skipping event: 78140, due to data points less than 250
Skipping event: 83585, due to data points less than 250
Skipping event: 86406, due to data points less than 250
Skipping event: 86445, due to data points less than 250
Skipping event: 87314, due to data points less than 250
Skipping event: 88063, due to data points less than 250
Skipping event: 88412, due to data points less than 250
Skipping event: 88416, due to data points less than 250
Skipping event: 89741, due to data points less than 250
Skipping event: 97188, due to data points less than 250
Skipping event: 102737, due to data points less than 250
Skipping event: 102740, due to data points less than 250
Skipping event: 110896, due to data points les

In [6]:
eventTensors = torch.stack(eventTensors, dim=0)
targets = torch.tensor(targets)
print("Event tensors shape: {}".format(eventTensors.shape))
print("Target tensors shape: {}".format(targets.shape))

Event tensors shape: torch.Size([65004, 4, 250])
Target tensors shape: torch.Size([65004])


In [7]:
torch.save(eventTensors, storeTensorsTo+"/eventTensors.pt")
torch.save(targets, storeTensorsTo+"/targets.pt")